# Target label definition

Describe the problem.

In [2]:
import pandas as pd
import os
import sys

# Append the path to access custom modules
if '../' not in sys.path:
    sys.path.append('../')

import src.modules.label_extraction as le
import src.modules.plotting as p
import importlib

from glob import glob

importlib.reload(le);
importlib.reload(p);

- `df_fsr`: contains freesurfers information
- `df_cdr`: contains data about subjects' visit from which it is possible to extract the target label

In [3]:
df_fsr = pd.read_csv('../data/csv/OASIS3_Freesurfer_output.csv')
df_cdr = pd.read_csv('../data/csv/OASIS3_UDSb4_cdr.csv')

- Remove from `df_fsr` the labels related to empty freesurfers

In [4]:
df_fsr = df_fsr[~df_fsr['MR_session'].isin(os.listdir('../data/empty'))]

- Define a new column in `df_cdr` and `df_fsr` that encapsule the temporal information

In [5]:
df_cdr['time'] = le.get_time_column(df_cdr['OASIS_session_label'])
df_fsr['time'] = le.get_time_column(df_fsr['MR_session'])

- Checking negative time columns for `df_cdr`:

In [6]:
df_cdr[df_cdr['time'] < 0].shape[0]

5

- Checking negative time columns for `df_fsr`:

In [7]:
df_fsr[df_fsr['time'] < 0].shape[0]

0

## Fixing negative time columns

- Clearly OAS30753 is a mistake: the cdr is null for this patient and we can't infer about its possible dementia
- The other ones looks like errors in typing, they will be fixed

In [8]:
df_cdr[df_cdr['time'] < 0]

,OASISID,OASIS_session_label,days_to_visit,age at visit,MMSE,memory,orient,judgment,commun,homehobb,...,dx2_code,dx3_code,dx4_code,dx5_code,dx1,dx2,dx3,dx4,dx5,time
1883,OAS30290,OAS30290_UDSb4_d-0002,-2,47.45,NaN,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,No dementia,.,.,.,.,-2
2116,OAS30330,OAS30330_UDSb4_d-0101,-101,80.53,24.0,0.5,0.5,0.0,0.0,0.5,...,NaN,NaN,NaN,NaN,"DLBD, primary",.,.,.,.,-101
2462,OAS30380,OAS30380_UDSb4_d-0015,-15,61.39,28.0,0.5,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,uncertain dementia,Active Other neurol/med diagnoses,.,.,.,-15
4914,OAS30753,OAS30753_UDSb4_d-39520,-39520,-47.25,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-39520
5667,OAS30851,OAS30851_UDSb4_d-0001,-1,73.49,NaN,0.5,0.0,0.5,0.5,0.5,...,NaN,NaN,NaN,NaN,"Non DAT, Other primary",.,.,.,.,-1


- Remove the error instance

In [9]:
df_cdr = df_cdr.drop(index=4914)
df_fsr = df_fsr.drop(index=df_fsr[df_fsr['Subject'] == 'OAS30753'].index)

- Fix the column and subject label names

In [9]:
df_cdr[['OASIS_session_label', 'time']] = le.fix_negative_time_label(df_cdr, 'time', 'OASIS_session_label')

In [11]:
df_cdr[['OASIS_session_label', 'OASISID']].to_csv('list2.csv', index=False)
df_fsr[['MR_session', 'Subject']].to_csv('list1.csv', index=False)

In [11]:
df_matched = pd.read_csv('../data/csv/out.csv')

In [13]:
df_matched = df_matched.drop(index=df_matched[df_matched['OASIS_session_label_list2'].isna()].index)

In [16]:
df_matched

,MR_session,Subject,Day,OASIS_session_label_list2,Subject_list2,Day_list2
0,OAS30001_MR_d0129,OAS30001,129,OAS30001_UDSb4_d0000,OAS30001,0.0
1,OAS30001_MR_d0757,OAS30001,757,OAS30001_UDSb4_d0722,OAS30001,722.0
3,OAS30001_MR_d3132,OAS30001,3132,OAS30001_UDSb4_d3025,OAS30001,3025.0
4,OAS30001_MR_d3746,OAS30001,3746,OAS30001_UDSb4_d3675,OAS30001,3675.0
5,OAS30001_MR_d4467,OAS30001,4467,OAS30001_UDSb4_d4349,OAS30001,4349.0
...,...,...,...,...,...,...
2471,OAS31470_MR_d0007,OAS31470,7,OAS31470_UDSb4_d0000,OAS31470,0.0
2472,OAS31471_MR_d0059,OAS31471,59,OAS31471_UDSb4_d0000,OAS31471,0.0
2473,OAS31472_MR_d0045,OAS31472,45,OAS31472_UDSb4_d0000,OAS31472,0.0
2474,OAS31473_MR_d0136,OAS31473,136,OAS31473_UDSb4_d0000,OAS31473,0.0


In [ ]:
df_matched = df_matched.merge(right=df_cdr, left_on='OASIS_session_label_list2', right_on='OASIS_session_label')


In [28]:
df_matched = df_matched.merge(right=df_fsr, left_on='MR_session', right_on='MR_session')

In [21]:
df_matched['CDRTOT'] = le.align_labels(df_matched, 'OASISID', 'CDRTOT')

OAS30139
OAS30173
OAS30181
OAS30185
OAS30206
OAS30208
OAS30241
OAS30342
OAS30369
OAS30418
OAS30445
OAS30452
OAS30479
OAS30596
OAS30597
OAS30612
OAS30667
OAS30810
OAS30818
OAS30867
OAS30896
OAS30959
OAS30978
OAS31001
OAS31024
OAS31037
OAS31043
OAS31104
OAS31111
OAS31127


In [23]:
df_matched['CDRTOT'] = df_matched['CDRTOT'].map(lambda x: 'Cognitevely-normal' if x == 0 else 'Early-stage' if x == .5 else 'Demented')

In [29]:
df_matched['CDRTOT'].value_counts()

CDRTOT
Cognitevely-normal    1694
Early-stage            353
Demented               104
Name: count, dtype: int64

- Perform a final check

In [25]:
f = os.path.join('..', 'data', 'freesurfers')

left_hippo_files = sorted(glob(os.path.join(f, '*', 'mri', '*Left-Hippocampus.mgz')))
right_hippo_files = sorted(glob(os.path.join(f, '*', 'mri', '*Right-Hippocampus.mgz')))

l1 = [s.split("\\")[3] for s in left_hippo_files]
l2 = [s.split("\\")[3] for s in right_hippo_files]

print(f"The number of elements NOT in common is: {set.difference(set(l1), set(l2))}")

The number of elements NOT in common is: set()


In [18]:
df_fsr.shape

(2475, 207)

In [30]:
df_matched[[
    'Subject', 'MR_session', 'TOTAL_HIPPOCAMPUS_VOLUME', 'Left-Hippocampus_volume', 
    'lh_parahippocampal_thickness', 'lh_parahippocampal_volume', 'rh_parahippocampal_volume',
    'rh_parahippocampal_thickness', 'Right-Hippocampus_volume', 'label', 'CDR',
]].to_csv('../data/csv/final_dataset.csv')

KeyError: "['Subject', 'label', 'CDR'] not in index"

In [26]:
df = pd.read_csv('../data/csv/final_dataset.csv')
df2 = df_cdr[['OASISID', 'CDRTOT', 'dx1', 'time']]

In [27]:
df2['CDRTOT'].value_counts()

CDRTOT
0.0    5678
0.5    2203
1.0     569
2.0     156
3.0      19
Name: count, dtype: int64